In [8]:
!pip install gmdh
!pip install numpy==1.24.4

In [9]:
import warnings
warnings.filterwarnings("ignore", category=ImportWarning)
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from gmdh import Combi, Mia
from sklearn.datasets import fetch_california_housing

In [10]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from gmdh import Combi, Mia
import pandas as pd

# 1) Загрузка и предобработка данных
print("1) Загрузка и предобработка данных...")
california = fetch_california_housing()
df = pd.DataFrame(california.data, columns=california.feature_names)
df['Price'] = california.target  # Добавляем целевую переменную (цена)

# Разделение данных на признаки (X) и целевую переменную (y)
X = df.drop(columns=['Price'])
y = df['Price']

# Масштабирование признаков
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Разделение данных (80% для обучения, 20% для теста)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, train_size=0.8, random_state=42)
print(f" → Train shape: {X_train.shape}, Test shape: {X_test.shape}")

# Подготовка данных для GMDH (немасштабированные для совместимости)
X_train_gmdh, X_test_gmdh, y_train_gmdh, y_test_gmdh = train_test_split(X, y, train_size=0.8, random_state=42)
X_train_np = X_train_gmdh.to_numpy(dtype=float)
X_test_np = X_test_gmdh.to_numpy(dtype=float)
y_train_np = y_train_gmdh.to_numpy(dtype=float).ravel()
y_test_np = y_test_gmdh.to_numpy(dtype=float).ravel()

print("\n4) Обучение GMDH на полной выборке...")
try:
    gmdh_linear = Combi()
    gmdh_linear.fit(X_train_np, y_train_np)
    print(" GMDH Linear (Combi) trained")
except Exception as e:
    print(f"Ошибка в GMDH Linear (Combi): {e}")
    gmdh_linear = None

# Обучение GMDH Nonlinear (Mia)
try:
    gmdh_nonlinear = Mia()
    gmdh_nonlinear.fit(X_train_np, y_train_np)
    print(" GMDH Nonlinear (Mia) trained")
except Exception as e:
    print(f"Ошибка в GMDH Nonlinear (Mia): {e}")
    gmdh_nonlinear = None

1) Загрузка и предобработка данных...
 → Train shape: (16512, 8), Test shape: (4128, 8)

4) Обучение GMDH на полной выборке...
 GMDH Linear (Combi) trained
 GMDH Nonlinear (Mia) trained


In [11]:
print("\n2) Обучение StackingRegressor...")
stack_model = StackingRegressor(
    estimators=[
        ('rf', RandomForestRegressor(n_estimators=50, random_state=42)),
        ('gb', GradientBoostingRegressor(n_estimators=50, random_state=42))
    ],
    final_estimator=LinearRegression()
)

stack_model.fit(X_train, y_train)
print(" Stacking trained")


2) Обучение StackingRegressor...
 Stacking trained


In [12]:
print("\n3) Обучение MLPRegressor...")
mlp_model = MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=1000, random_state=42)
mlp_model.fit(X_train, y_train)
print(" MLP trained")


3) Обучение MLPRegressor...
 MLP trained


In [13]:
# 5. Оценка качества
print("\n5) Оценка качества всех моделей:")
models = {
    'Stacking': (stack_model, X_test, y_test),
    'MLP': (mlp_model, X_test, y_test),
    'GMDH-Linear': (gmdh_linear, X_test_np, y_test_np),
    'GMDH-Nonlinear': (gmdh_nonlinear, X_test_np, y_test_np)
}

results = []
for name, (model, X_eval, y_eval) in models.items():
    if model is None:
        print(f" • {name}: Пропущен из-за ошибки")
        results.append({'Model': name, 'MAE': np.nan, 'R2': np.nan})
        continue
    y_pred = model.predict(X_eval)
    mae = mean_absolute_error(y_eval, y_pred)
    r2 = r2_score(y_eval, y_pred)
    results.append({'Model': name, 'MAE': mae, 'R2': r2})
    print(f" • {name}: MAE = {mae:.2f}, R² = {r2:.4f}")


5) Оценка качества всех моделей:
 • Stacking: MAE = 0.33, R² = 0.8034
 • MLP: MAE = 0.34, R² = 0.8037
 • GMDH-Linear: MAE = 0.53, R² = 0.5752
 • GMDH-Nonlinear: MAE = 0.59, R² = 0.5144
